## Sources

In [ ]:
# https://medium.com/ai-techsystems/insect-classification-2c89e7398ec4

In [ ]:
# https://www.linkedin.com/pulse/using-ai-recognize-kissing-bugs-mobile-phone-images-lorenzo-pattori/

In [ ]:
# https://web.stanford.edu/~nanbhas/blog/sigmoid-softmax/#:~:text=And%20the%20sigmoid%20can%20now,belongs%20to%20the%20negative%20class.

## Import libraries and set configurations

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks

In [10]:
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                        inter_op_parallelism_threads=2, 
                        allow_soft_placement=True,
                        device_count = {'CPU': 1})

session = tf.compat.v1.Session(config=config)

In [11]:
# Directory where images are present
valid_dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/valid_ds'
train_dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/train_ds'
num_fldrs = 2
target_size = (224,224)

In [12]:
# dictionary of labels
insect_names = {'1':"triatomineo",'2':"outro_inseto"}

## Get and load data

In [13]:
def datapreprocessing(train_dir, valid_dir, batch_size):
    from tensorflow.keras.utils import image_dataset_from_directory

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        image_size=target_size,
        batch_size=batch_size,
        shuffle=True,
        )

    valid_ds = image_dataset_from_directory(
        directory=valid_dir,
        image_size=target_size,
        batch_size=batch_size,
        shuffle=False,
        )
    
    return train_ds, valid_ds

In [ ]:
batch_size = 32

train_ds, valid_ds = datapreprocessing(train_dir, valid_dir, batch_size=batch_size)

## Augmentation

In [15]:
data_random_Flip = tf.keras.layers.RandomFlip("horizontal_and_vertical")
data_random_rotation = tf.keras.layers.RandomRotation(0.4)

In [16]:
def augmentation(ds):
    aug1 = ds.map(lambda x, y: (data_random_Flip(x), y))
    aug2 = ds.map(lambda x, y: (data_random_rotation(x), y))
    
    ds = ds.concatenate(aug1)
    ds = ds.concatenate(aug2)
 
    return ds 

In [17]:
aug_train_ds = augmentation(train_ds)

In [18]:
print(f'The training dataset was updated from {len(list(train_ds))} to {len(list(aug_train_ds))} image batchs')

2024-05-15 09:24:54.440433: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [705]
	 [[{{node Placeholder/_4}}]]
2024-05-15 09:24:54.440626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [705]
	 [[{{node Placeholder/_4}}]]
2024-05-15 09:24:55.374053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_41' with dtype int32 and shape [705]
	 [

The training dataset was updated from 23 to 69 image batchs


## CNN model (ResNet18)

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras.activations import relu
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras import layers as Layers


class ResBlock(Model):
    def __init__(self, channels, stride=1):
        super(ResBlock, self).__init__(name='ResBlock')
        self.flag = (stride != 1)
        self.conv1 = Conv2D(channels, 3, stride, padding='same')
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2D(channels, 3, padding='same')
        self.bn2 = BatchNormalization()
        self.relu = ReLU()
        if self.flag:
            self.bn3 = BatchNormalization()
            self.conv3 = Conv2D(channels, 1, stride)

    def call(self, x):
        x1 = self.conv1(x)
        x1 = self.bn1(x1)
        x1 = self.relu(x1)
        x1 = self.conv2(x1)
        x1 = self.bn2(x1)
        if self.flag:
            x = self.conv3(x)
            x = self.bn3(x)
        x1 = Layers.add([x, x1])
        x1 = self.relu(x1)
        return x1


class ResNet18(Model):
    def __init__(self):
        super(ResNet18, self).__init__(name='ResNet18')
        self.conv1 = Conv2D(64, 7, 2, padding='same')
        self.bn = BatchNormalization()
        self.relu = ReLU()
        self.mp1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding="same")

        self.conv2_1 = ResBlock(64)
        self.conv2_2 = ResBlock(64)

        self.conv3_1 = ResBlock(128, 2)
        self.conv3_2 = ResBlock(128)

        self.conv4_1 = ResBlock(256, 2)
        self.conv4_2 = ResBlock(256)

        self.conv5_1 = ResBlock(512, 2)
        self.conv5_2 = ResBlock(512)

        self.pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(2, name="output")
        
    def call(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.mp1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.conv4_1(x)
        x = self.conv4_2(x)

        x = self.conv5_1(x)
        x = self.conv5_2(x)

        x = self.pool(x)
        x = self.flat(x)
        x = self.fc(x)
        
        return x


model = ResNet18()
model.build(input_shape=(1, 224, 224, 3))
model.summary()

Model: "ResNet18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_103 (Conv2D)         multiple                  9472      
                                                                 
 batch_normalization_103 (Ba  multiple                 256       
 tchNormalization)                                               
                                                                 
 re_lu_49 (ReLU)             multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 ResBlock (ResBlock)         multiple                  74368     
                                                                 
 ResBlock (ResBlock)         multiple                  743

In [ ]:
from tensorflow.keras import datasets, layers, models, losses, Model

In [ ]:
base_model = tf.keras.applications.ResNet50(weights = None, include_top = False, input_shape = (150,150,3))
for layer in base_model.layers:
  layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(1)(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
def insectclf(input_shape):

    from tensorflow import keras as ks

    model = ks.models.Sequential()

    model.add(ks.layers.Rescaling(1./255))

    model.add(ks.layers.Conv2D(16,(3,3),
                               strides=1,
                               activation="relu",
                               padding='valid',
                               name="layer1",
                               input_shape=input_shape))
    model.add(ks.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(ks.layers.Conv2D(32,(3,3),
                               strides=1,
                               activation="relu",
                               padding="valid",
                               name="layer2"))
    model.add(ks.layers.MaxPooling2D(pool_size=(2,2)))
    
    model.add(ks.layers.Conv2D(64,(3,3),
                               strides=1,
                               activation="relu",
                               padding="valid",
                               name="layer3"))
    model.add(ks.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(ks.layers.Conv2D(128,(3,3),
                               strides=1,
                               activation="relu",
                               padding="valid",
                               name="layer4"))
    model.add(ks.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(ks.layers.Conv2D(256,(3,3),
                               strides=1,
                               activation="relu",
                               padding='valid',
                               name="layer5",
                               input_shape=input_shape))
    model.add(ks.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(ks.layers.Flatten())

    model.add(ks.layers.Dense(256,activation="relu", name="layer6"))
    model.add(ks.layers.Dropout(0.5))

    model.add(ks.layers.Dense(1, name="output"))

    #model.summary()

    return model

In [ ]:
model = insectclf(target_size)

In [ ]:
def insectclf(input_shape):
    from tensorflow import keras as ks
    #from tensorflow.keras import regularizers
    model = ks.models.Sequential()
    #building architecture
    #Adding layers
    model.add(ks.layers.Rescaling(1./255))
    model.add(ks.layers.Conv2D(300,(3,3),
                               strides=1,
                               activation="relu",
                               padding='valid',
                               name="layer1",
                               input_shape=input_shape))
    model.add(ks.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(ks.layers.Flatten())
    model.add(ks.layers.Dense(100,activation="relu", name="layer2"))
    model.add(ks.layers.Dropout(0.5))
    model.add(ks.layers.Dense(100,activation="relu", name="layer3"))
    model.add(ks.layers.Dropout(0.5))
    model.add(ks.layers.Dense(1, name="output"))
    #model.summary()

    return model

In [ ]:
# Get input shape
input_shape = target_size
#Build Model
model = insectclf(input_shape)

## Training

In [20]:
def compiler(model,train_ds,valid_ds,epchs,lr):
    early_stopping = ks.callbacks.EarlyStopping(monitor='val_accuracy',patience=10,
                                         verbose=1,restore_best_weights=True)
    #red_lr= ks.callbacks.ReduceLROnPlateau(monitor='val_accuracy',patience=20,verbose=1,factor=0.1)
    opt = ks.optimizers.Adam(learning_rate=lr)

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      optimizer=opt,
                      metrics=["accuracy"])
    history = model.fit(train_ds,
                        epochs=epchs,
                        callbacks=[early_stopping],
                        validation_data=valid_ds
                        )
    #Visualise curves
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='valid_acc')

    plt.title('lrate='+str(lr), pad=-50)
    plt.legend()
    plt.grid(True)
    return model,history

In [ ]:
model = compiler(model,aug_train_ds,valid_ds,7,lr=0.001)

## Save model

In [16]:
model[0].save(filepath='/home/reginaldo/Documents/ime_5.1/fiocruz/results/cnn_model.keras')

## Load model

In [17]:
model = tf.keras.models.load_model(filepath='/home/reginaldo/Documents/ime_5.1/fiocruz/results/cnn_model.keras')

In [7]:
def prediction_func(modelo_dir, imagem_dir, save_imagem_dir):
    model = model_loader.load_model(modelo_dir)

    target_size = (224, 224)
    image_directory = imagem_dir
    img_input = Image.open(image_directory)
    img_output = remove(img_input)
    img_save_directory = save_imagem_dir
    img_output.save(img_save_directory)
    
    image = image_loader.load_img(img_save_directory, target_size=target_size)
    image_arr = np.array([image])
    
    output = np.array(activation_function.softmax(model.predict(image_arr)))[0] * 100

    nivel_certeza = ''
    
    if output[1] < 55.0:
        nivel_certeza = 'BAIXA'
    elif (output[1] >= 55.0 and output[1] < 70.0):
        nivel_certeza = 'MÉDIA'
    elif (output[1] >= 70.0 and output[1] < 85.0):
        nivel_certeza = 'MODERADA'
    elif (output[1] >= 85.0 and output[1] < 95.0):
        nivel_certeza = 'ALTA'
    elif output[1] >= 95.0:
        nivel_certeza = 'ALTÍSSIMA'

    print(f'É um barbeiro com uma certeza de aprox. {round(output[1], 1)}%' if output[1] > output[0] else 
          f'Não é um barbeiro com uma certeza de aprox. {round(output[0], 1)}%\n\n')
    print(f'Sua foto demonstra uma {nivel_certeza} chance de ser um barbeiro.\n', 
      'A identificação será confirmada por um especialista e em breve voce receberá uma mensagem de confirmação.\n\n',
      'Siga as recomendações abaixo:\n\n',
      '1 - Proteja suas mãos usando um saco ou sacola plástica, assim você irá evitar o contato direto com o inseto;\n',
      '2 - Nunca tente matar ou esmagar o inseto, essa ação pode levar ao aumento do risco.\n',
      '3 - Recolha cuidadosamente o isento e coloque em um recipiente seco e tampado. Caso o inseto esteja vivo, realizar pequenos furos na tampa para possibilitar a ventilação.\n',
      'Caso o inseto ainda esteja no local deve ser levado, preferencialmente vivo, dentro de um frasco plástico lacrado para o PIT (Posto de identificação de Triatomíneos) mais próximo de sua residência\n', 
      '(veja a localização dos PITs em:  https://chagas.fiocruz.br/entregue-pessoalmente).\n\n',  
      'Caso não encontre um PIT próximo, oriento a entrar em contato com o órgão de saúde do seu município informando sobre o encontro do inseto, para que sejam tomadas as providências cabíveis.\n', 
      'O inseto não deve ser manuseado com as mãos desprotegidas.')

In [ ]:
model_dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/best_result/model_CNN1_com_aug.keras'
img_dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/fiocruz_reginaldo/dataset_CNN-A/triatomineo/00000005.jpg'
save_img_dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/fiocruz_reginaldo/dataset_CNN-A/teste.png'

prediction_func(model_dir, img_dir, save_img_dir)

## Inferência de triatomineos da validação

In [ ]:
import os

dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/valid_ds/triatomineo'
target_size = (224, 224)
results = []

# Definir opções de impressão do NumPy para formatar números decimais completos
np.set_printoptions(precision=2, suppress=True)
for img in os.listdir(dir):
  image = image_loader.load_img(dir+'/'+img, target_size=target_size)
  image_arr = np.array([image])

  output = np.array(activation_function.softmax(model.predict(image_arr)))[0] * 100
  results.append(output)

In [ ]:
erros = 0
for count, img in enumerate(os.listdir(dir)):
    #print(img, f'É um barbeiro com uma certeza de aprox. {results[count][1]}%' if results[count][1] > results[count][0] else 
    #  f'Não é um barbeiro com uma certeza de aprox. {results[count][0]}%')
    print(f'Inferência incorreta de {img} com {results[count]}' if results[count][1] <= results[count][0] else None)
    if results[count][1] <= results[count][0]:
        erros += 1
soma = len(os.listdir(dir))
print(f'\nPorcentagem de acerto é de {((soma - erros) / soma) * 100}%')

## Inferência de outros insetos da validação

In [ ]:
dir = '/home/reginaldo/Documents/ime_5.1/fiocruz/valid_ds/outro_inseto'
target_size = (224, 224)
results = []

# Definir opções de impressão do NumPy para formatar números decimais completos
np.set_printoptions(precision=2, suppress=True)
for img in os.listdir(dir):
  image = image_loader.load_img(dir+'/'+img, target_size=target_size)
  image_arr = np.array([image])

  output = np.array(activation_function.softmax(model.predict(image_arr)))[0] * 100
  results.append(output)

In [ ]:
erros = 0
for count, img in enumerate(os.listdir(dir)):
    #print(img, f'É um barbeiro com uma certeza de aprox. {results[count][1]}%' if results[count][1] > results[count][0] else 
    #  f'Não é um barbeiro com uma certeza de aprox. {results[count][0]}%')
    print(f'Inferência incorreta de {img} com {results[count]}' if results[count][0] <= results[count][1] else None)
    if results[count][0] <= results[count][1]:
        erros += 1
soma = len(os.listdir(dir))
print(f'Porcentagem de acerto é de {((soma - erros) / soma) * 100}%')

## Confusion's Matrix

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def criar_matriz_confusao(verdadeiro_positivo_A, verdadeiro_negativo_B, total_A, total_B):
    falso_positivo_B = total_B - verdadeiro_negativo_B
    falso_negativo_A = total_A - verdadeiro_positivo_A
    
#    matriz_confusao = np.array([[verdadeiro_positivo_A / total_A, falso_negativo_A / total_A], 
#                                [falso_positivo_B / total_B, verdadeiro_negativo_B / total_B]])

    matriz_confusao = np.array([[verdadeiro_positivo_A, falso_negativo_A], 
                                [falso_positivo_B, verdadeiro_negativo_B]])

#    matriz_confusao_porcentagem = matriz_confusao * 100
#    matriz_confusao_porcentagem = np.round(matriz_confusao_porcentagem, 2)

    return matriz_confusao

# Dados fornecidos
verdadeiro_positivo_A = 61
verdadeiro_negativo_B = 59
total_A = 66
total_B = 66

# Chamada da função para criar a matriz de confusão
matriz = criar_matriz_confusao(verdadeiro_positivo_A, verdadeiro_negativo_B, total_A, total_B)

# Criar o gráfico da matriz de confusão
fig, ax = plt.subplots()
im = ax.imshow(matriz, cmap='viridis')

# Adicionar textos dentro dos quadrados
for i in range(len(matriz)):
    for j in range(len(matriz[i])):
        text = ax.text(j, i, matriz[i, j],
                       ha="center", va="center", color="black", fontsize=18, bbox={'facecolor': 'white', 'alpha': 1.0, 'pad': 5})

# Adicionar rótulos aos eixos
ax.set_xticks(np.arange(len(matriz)))
ax.set_yticks(np.arange(len(matriz)))
ax.set_xticklabels(['Triatomineo', 'Outro inseto'])
ax.set_yticklabels(['Triatomineo', 'Outro inseto'])
ax.set_xlabel('Predição')
ax.set_ylabel('Verdadeiro valor')

# Adicionar barra de cores
#cbar = ax.figure.colorbar(im, ax=ax)
#cbar.ax.set_ylabel("Contagem", rotation=-90, va="bottom")

# Rotacionar os rótulos dos eixos para melhor visualização
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Adicionar título
ax.set_title("Matriz de Confusão")

# Mostrar o gráfico
plt.show()
